In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
#read in; had to use converters b/c the truck names had mixed types
df = pd.read_csv("20180101.csv", parse_dates=['date_fixed'], converters={'truck_name': str})

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39784 entries, 0 to 39783
Data columns (total 5 columns):
truck_name    39784 non-null object
date_fixed    39784 non-null datetime64[ns]
address       38861 non-null object
longitude     39784 non-null float64
latitude      39784 non-null float64
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 1.5+ MB


### Dates are Pandas Timestamps

In [4]:
df.loc[1700,'date_fixed']

Timestamp('2018-01-01 16:25:29')

### Use Timedelta method to shift time

In [5]:
df.loc[1700,'date_fixed'] - pd.Timedelta(hours=5)

Timestamp('2018-01-01 11:25:29')

In [6]:
#stripping white space from the truck names
df['truck_name'] = df['truck_name'].str.strip()

In [7]:
#sorted by truck then date/time
df = df.sort_values(by=['truck_name','date_fixed'],na_position='first')

In [8]:
df['date_fixed'] = df['date_fixed'] - pd.Timedelta(hours=5)

In [28]:
#test subset 
df4= df3[15000:15200]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39784 entries, 8427 to 34783
Data columns (total 5 columns):
truck_name    39784 non-null object
date_fixed    39784 non-null datetime64[ns]
address       38861 non-null object
longitude     39784 non-null float64
latitude      39784 non-null float64
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 1.8+ MB


In [11]:
df.head()

,truck_name,date_fixed,address,longitude,latitude
8427,178,2018-01-01 05:15:20,1200 CANAL ST,-76.108890,43.055110
8542,178,2018-01-01 05:15:22,1200 CANAL ST,-76.108890,43.055110
8547,178,2018-01-01 05:16:22,1200 CANAL ST,-76.108890,43.055110
8552,178,2018-01-01 05:17:22,1200 CANAL ST,-76.108805,43.055128
8648,178,2018-01-01 05:18:22,1200 CANAL ST,-76.108779,43.055191


In [12]:
# get the manhattan distance between consecutive data points
df['manh_dist'] = abs(df.longitude - df.longitude.shift()) + abs(df.latitude - df.latitude.shift())

In [13]:
df[df.manh_dist > .01].count()

truck_name    239
date_fixed    239
address       236
longitude     239
latitude      239
manh_dist     239
dtype: int64

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39784 entries, 8427 to 34783
Data columns (total 6 columns):
truck_name    39784 non-null object
date_fixed    39784 non-null datetime64[ns]
address       38861 non-null object
longitude     39784 non-null float64
latitude      39784 non-null float64
manh_dist     39783 non-null float64
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 2.1+ MB


In [15]:
#how many data points per truck
df.groupby('truck_name').size()

truck_name
178     1722
179     1682
182        6
206      143
207        1
208        1
210     1285
211        1
212     1288
216     4065
251     3811
253        1
254       47
257        1
258      626
259       27
260.       1
261     3341
262        1
263     3272
264     2364
265     3461
266     2004
267        1
268      686
269        1
270        6
271     2398
282     2203
284     2234
285     2667
73       437
dtype: int64

# Left off here. Create an 'ID' column, which is a mapping from a point to a road segment

In [16]:
objID_to_bnd_box = pickle.load( open( "ID_to_bound_box", "rb" ) )

In [21]:
#test iterrows; delete eventually
tst2 = []
for index, row in df.iterrows():
    tst2.append([index,row])
    

In [30]:
# an index and row
tst2[10]

[8873, truck_name                    178
 date_fixed    2018-01-01 05:24:22
 address             1200 CANAL ST
 longitude                -76.1087
 latitude                  43.0552
 manh_dist                5.44e-05
 Name: 8873, dtype: object]

In [36]:
# a row (as a Series)
tst2[10][1]

truck_name                    178
date_fixed    2018-01-01 05:24:22
address             1200 CANAL ST
longitude                -76.1087
latitude                  43.0552
manh_dist                5.44e-05
Name: 8873, dtype: object

In [38]:
tst2[1000][1].latitude

43.0432602080756

In [46]:
objID_to_bnd_box[10]

[[-76.1443080869106, 43.0825212790999], [-76.1449873163995, 43.0832022957928]]

## Left off here. How to incrementally add values to the not yet created 'ID' column?

In [39]:
for idx,row in df.iterrows():
    coords = [row.longitude, row.latitude]
    for k,v in objID_to_bnd_box.items():
        if (row.longitude > min(v[0][0],vv[1][0]) and row.longitude < max(v[0][0],vv[1][0])
                                                                      and row.latitude > min(v[0][1],vv[1][1])
                                                                      and row.latitude < max(v[0][1],vv[1][1]) ):
            df['ID'] = 
    

# Below here is not used

In [116]:
#NOT USED; generate test dataframe
test_row_iter = pd.DataFrame(np.random.randn(20,3),columns=list('ABC'))

In [118]:
#NOT USED
#for testing: set column C to absolute values
test_row_iter['C'] = abs(test_row_iter['C'])

In [85]:
#NOT USED
#how to iterate over rows and set a boolean value based on a conditional
for i in range(1,len(test_row_iter)):
    if test_row_iter.loc[i, 'C'] > 0:
        test_row_iter.loc[i,'D'] = True

In [120]:
#NOT USED
#set first boolean of the 'Keep' column to 'True', since we always want to keep the first data point
test_row_iter.loc[0,'Keep'] = True
#create a boolean column based on an the accumulated value of a column
threshhold = 2.
cum_dist = 0.
for i in range(1,len(test_row_iter)):
    if test_row_iter.loc[i, 'C'] + cum_dist > threshhold:
        test_row_iter.loc[i,'Keep'] = True
        cum_dist = 0
    else:
        test_row_iter.loc[i,'Keep'] = False
        cum_dist += test_row_iter.loc[i, 'C']
            

In [127]:
#NOT USED
test_row_iter = test_row_iter[test_row_iter.Keep == True]